The code in this notebook is a reimplementation of the classical computing version of the code described in the paper: https://arxiv.org/abs/2304.09224

In [2]:
%pip install torch torchvision
%pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 30.8 MB/s eta 0:00:00


In [3]:
import torch
import torchvision
from torchvision import transforms, datasets
from torchvision.transforms import ToTensor
import torch.optim as optim
import torch.nn as nn
import pennylane as qml

# Data Preparation and Loading

Use the MNIST dataset: https://en.wikipedia.org/wiki/CIFAR-10

In [4]:
# Download and load MNIST dataset

train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True,
)
test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor()
)

100%|██████████| 9912422/9912422 [00:00<00:00, 109135205.23it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 22825644.21it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 32306380.08it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 14848424.60it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



# Creating the CNN Architecture

The CNN architecture is defined with two convolutional layers, max pooling layers, and fully connected layers. The forward method specifies how data flows through the network. This architecture is suitable for image classification tasks and can be modified to suit your specific project requirements.

In [5]:
import torch.nn as nn
import torch.nn.functional as F

# Define a simple CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Convolutional layer 1 with 1 input channels (for greyscale images), 16 output channels, and 5x5 kernel
        self.conv1 = nn.Conv2d(1, 16, 5, stride=1, padding=2)
        # Batch normalization after convolutional layer 1
        self.bn1 = nn.BatchNorm2d(16)
        # Max pooling layer with a 2x2 window
        self.pool = nn.MaxPool2d(2, 2)
        # Convolutional layer 2 with 16 input channels (from the previous layer), 32 output channels, and 5x5 kernel
        self.conv2 = nn.Conv2d(16, 32, 5, stride=1, padding=2)
        # Batch normalization after convolutional layer 2
        self.bn2 = nn.BatchNorm2d(32)
        # Fully connected layers
        self.fc1 = nn.Linear(32 * 7 * 7, 120)
        self.fc2 = nn.Linear(120, 20)
        self.fc3 = nn.Linear(20, 10)

    def forward(self, x):
        # Propagate the input through the CNN layers
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        # Flatten the output from the convolutional layers
        x = x.view(-1, 32 * 7 * 7)
        # Pass the output to the quantum layer
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Train the CNN

The CNN model is initialised, the loss function and optimizer are set up, and data loaders for training and validation data are created. The training loop iterates through the dataset for a specified number of epochs, performing forward and backward passes to update the model’s parameters.

In [6]:
dataset  = train_data

# Initialize your CNN model
cnn = Net()
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = torch.optim.SGD(cnn.parameters(), lr=0.001, momentum=0.9)  # Stochastic Gradient Descent optimizer
# Split your data into training and validation sets
train_size = int(0.8 * len(dataset))
train_set, val_set = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=4, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=4, shuffle=False)
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()  # Zero the parameter gradients to avoid accumulation
        outputs = cnn(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update the model parameters
print('Finished Training')

Finished Training


# Evaluating the Model

Set the model to evaluation mode, use it to make predictions on the validation dataset, and calculate the accuracy of the model.

In [7]:
correct = 0
total = 0
# Set the model to evaluation mode
cnn.eval()
with torch.no_grad():
    for data in val_loader:
        images, labels = data
        outputs = cnn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy on the validation set: {100 * correct / total:.2f}%')

Accuracy on the validation set: 99.20%
